本转换脚本适用于微信小程序 **v1.3.2**、Web App 后端 **v0.2.0**

**本脚本会抹除后端数据库中所有数据！！！！！！**如需要，请提前做好备份！！！

首先需要将微信小程序云数据库的 `activity_info`、`album_info`、`comment_info` 和 `user_info` 表分别导出为 `activity_info.json`、`album_info.json`、`comment_info.json` 和 `user_info.json`，放在该 Jupyter Notebook 的同一目录。

请提前做好对 `user_info.json` 的数据清洗。

In [1]:
# 设置 superuser 的信息，请及时修改
superuser_name = '刘俨晖'
superuser_email = 'lyh543@outlook.com'
superuser_password = 'superuser'

In [2]:
import os
# 设置 Django 项目的路径
DJANGO_PROJECT_PATH = r"C:\Tools\Git\uestcmsc-webapp\uestcmsc_webapp_backend"
# 备份当前路径
try:
    os.chdir(WX_DATA_PATH)
except NameError:
    WX_DATA_PATH = os.path.abspath('.')

In [3]:
import json
def read_wx_json(filename: str) -> list[dict]: 
    li = []
    with open(filename, encoding='utf-8') as f:
        for line in f:
            if not line.startswith('//'):   # 支持注释
                li.append(json.loads(line))
    return li

wx_activity_info = read_wx_json('activity_info.json')
wx_gallery_info = read_wx_json('album_info.json')
wx_comment_info = read_wx_json('comment_info.json')
wx_user_info = read_wx_json('user_info.json')

In [4]:
import django
os.chdir(DJANGO_PROJECT_PATH)
os.environ["DJANGO_SETTINGS_MODULE"]="uestcmsc_webapp_backend.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [5]:
from users.models import User, UserProfile
from activities.models import Activity
from activities_comments.models import Comment
# from gallery.models import Picture

In [6]:
from datetime import date, time, datetime
def str2date(string: str) -> date:
    return datetime.strptime(string, '%Y-%m-%d').date()
def str2time(string: str) -> time:
    return datetime.strptime(string, '%H:%M').time()
def str2datetime(string_date: str, string_time: str) -> datetime:
    return datetime.combine(str2date(string_date), str2time(string_time))

In [7]:
# 用户部分

User.objects.all().delete()

webapp_user = dict()
for wx_user in wx_user_info:

    u = User(
        username=wx_user['_id'],
        is_staff=wx_user['is_admin'],
        first_name=wx_user['username'],
        is_superuser=wx_user['can_grant_admin'],
        date_joined=str2date(wx_user['register_date'])
    )
    u.save()
    up = UserProfile(
        user=u,
        student_id=wx_user['student_id'],
        experience=wx_user['exp'])
    up.save()
    webapp_user[wx_user['_id']] = u

superuser = User.objects.filter(first_name=superuser_name)[0]
superuser.username = superuser_email
superuser.is_superuser = True
superuser.is_staff = True
from hashlib import md5
md5_password = md5(superuser_password.encode()).hexdigest()
superuser.set_password(md5_password)
superuser.save()


C:\Tools\Python\lib\site-packages\django\db\models\fields\__init__.py:1309: RuntimeWarning: DateTimeField User.date_joined received a naive datetime (2020-08-26 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s.%s received a naive datetime "
C:\Tools\Python\lib\site-packages\django\db\models\fields\__init__.py:1309: RuntimeWarning: DateTimeField User.date_joined received a naive datetime (2020-08-04 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s.%s received a naive datetime "
C:\Tools\Python\lib\site-packages\django\db\models\fields\__init__.py:1309: RuntimeWarning: DateTimeField User.date_joined received a naive datetime (2020-08-10 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s.%s received a naive datetime "
C:\Tools\Python\lib\site-packages\django\db\models\fields\__init__.py:1309: RuntimeWarning: DateTimeField User.date_joined received a naive datetime (2020-08-11 00:00:00) while time zone 

In [8]:
# 活动部分

webapp_activity = dict()
Activity.objects.all().delete()

for wx_activity in wx_activity_info:
    if wx_activity['is_hidden']:
        continue

    activity = Activity(
            datetime = str2datetime(wx_activity['date'], wx_activity['time']),
            location = wx_activity['location'],
            title = wx_activity['title'],
    )
    activity.save()
    webapp_activity[wx_activity['_id']] = activity

    for wx_presenter_id in wx_activity['presenter_list']:
        activity.presenter.add(webapp_user[wx_presenter_id])
    for wx_attender_id in wx_activity['check_in_list']:
        if wx_attender_id in webapp_user: # 可能有用户参加了活动，但没有绑定信息，只能忽略掉此次签到
            activity.attender.add(webapp_user[wx_attender_id])

C:\Tools\Python\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField Activity.datetime received a naive datetime (2020-09-20 20:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Tools\Python\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField Activity.datetime received a naive datetime (2020-09-13 21:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Tools\Python\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField Activity.datetime received a naive datetime (2020-09-16 20:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
C:\Tools\Python\lib\site-packages\django\db\models\fields\__init__.py:1367: RuntimeWarning: DateTimeField Activity.datetime received a naive datetime (2020-09-19 20:00:00) while tim

In [9]:
# 评论部分

webapp_comment = []
webapp_commentlike = []
for wx_comment in wx_comment_info:
    if 'reply' not in wx_comment:
        wx_comment['reply']=''
    comment = Comment(
        activity = webapp_activity[wx_comment['activity_id']],
        user = webapp_user[wx_comment['_openid']],
        text = wx_comment['comment_text'],
        reply = wx_comment['reply']
    )
    webapp_comment.append(comment)
    for wx_like_id in wx_comment['like_list']:
        webapp_commentlike.append(CommentLike(comment=comment, user=webapp_user[wx_like_id]))

KeyError: 'b4f7b7d25fa8a806005850a2362afc47'

In [9]:
# 将微信链接："cloud://uestc-msc-activities.7565-uestc-msc-activities-1302714164/album/1b64dd7b5f92dd3801e7dfc40da7bd69/1603627997528_o42P84g3DmWZIj-qWwPg7tRCT77o_0.jpg"
# 转为："https://7565-uestc-msc-activities-1302714164.tcb.qcloud.la/album/1b64dd7b5f92dd3801e7dfc40da7bd69/1603627997528_o42P84g3DmWZIj-qWwPg7tRCT77o_0.jpg"
def convert_wxcloud_url(wxcloud: str) -> str:
    return "233"

# 测试
convert_wxcloud_url("cloud://uestc-msc-activities.7565-uestc-msc-activities-1302714164/album/1b64dd7b5f92dd3801e7dfc40da7bd69/1603627997528_o42P84g3DmWZIj-qWwPg7tRCT77o_0.jpg") == "https://7565-uestc-msc-activities-1302714164.tcb.qcloud.la/album/1b64dd7b5f92dd3801e7dfc40da7bd69/1603627997528_o42P84g3DmWZIj-qWwPg7tRCT77o_0.jpg"

False

In [10]:
# 照片的转移还需要等等
webapp_picture = []
for wx_gallery in wx_gallery_info:
    if wx_gallery['_openid'] not in webapp_user: # 照片上传者没有注册
        webapp_picture.append(Picture(
            activity = webapp_activity[wx_gallery['album_id']],
            url = wx_gallery['url']
        ))
    else:
        webapp_picture.append(Picture(
            uploader = webapp_user[wx_gallery['_openid']],
            activity = webapp_activity[wx_gallery['album_id']],
            url = wx_gallery['url']
        ))

In [11]:
webapp_user
webapp_activity
webapp_attender
webapp_presenter
webapp_comment
webapp_commentlike
webapp_picture

[<Picture: cloud://uestc-msc-activities.7565-uestc-msc-activities-1302714164/album/b5416b755f5b5ac0019376fc5b89cd06/1600002165983_o42P84vBx_TXxg_sEIa2aC73b_l8.jpg>,
 <Picture: cloud://uestc-msc-activities.7565-uestc-msc-activities-1302714164/album/f1168d5b5f607d7a0004fa9d01cefd82/1600162787936_o42P84kmGnGB7oWgcqT0sDCOXoNU.jpg>,
 <Picture: cloud://uestc-msc-activities.7565-uestc-msc-activities-1302714164/album/ac5f38825f54975b01143cf22c3a9f35/1600605442097_o42P84vBx_TXxg_sEIa2aC73b_l8_0.jpg>,
 <Picture: cloud://uestc-msc-activities.7565-uestc-msc-activities-1302714164/album/ac5f38825f54975b01143cf22c3a9f35/1600605442102_o42P84vBx_TXxg_sEIa2aC73b_l8_1.jpg>,
 <Picture: cloud://uestc-msc-activities.7565-uestc-msc-activities-1302714164/album/ac5f38825f54975b01143cf22c3a9f35/1600605442104_o42P84vBx_TXxg_sEIa2aC73b_l8_2.jpg>,
 <Picture: cloud://uestc-msc-activities.7565-uestc-msc-activities-1302714164/album/8e5be7055f71d3fd00c05cc163597c49/1601368988278_o42P84rDYvFyPMTwHtIWPUn2fGr0_0.jpg>,
 <